# Mo's Algorithm

Very important [link](https://www.hackerearth.com/practice/notes/mos-algorithm/) to refer.
  
So basic idea is to sort the querry in intervals such that you don't have to move mo's right and mo's left not more than $n\sqrt{n}$.
And rule for that is.  
Rearrange all queries in a way we will call “Mo’s order”. It is defined like this: [L1, R1] comes earlier than [L2, R2] in Mo’s order if and only   if:  
   a) $\frac{L1}{BLOCK\_SIZE} < \frac{L2}{BLOCK\_SIZE}$  
   b) $\frac{L1}{BLOCK\_SIZE} = \frac{L2}{BLOCK\_SIZE}$ and $R1 < R2$  

[Here](https://codeforces.com/problemset/problem/86/D) is the question and below is the solution code.  

In [ ]:
import sys; #input = sys.stdin.readline

n, q = map(int, input().split())
lis = list(map(int, input().split()))
querry = []
for i in range(q): 
    a,b = map(int, input().split())
    querry.append((i, a, b))

D = int(n**0.5)
querry.sort(key=lambda x: (x[1]/D, x[2])) # sorting by the criteria
cnt = [0]*int(1e6)
ans = [0]*q
ml, mr = 0, -1
cur = 0
for i, l, r in querry:
    l-=1; r-=1
    while ml<l:
        p = cnt[lis[ml]]
        cnt[lis[ml]] -= 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur -= (2*p -1)*lis[ml]
        ml+=1
    while ml>l:
        ml-=1
        p = cnt[lis[ml]]
        cnt[lis[ml]] += 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur += (2*p+1)*lis[ml]
    while mr<r:
        mr+=1
        p = cnt[lis[mr]]
        cnt[lis[mr]] += 1
        # cur += (cnt[lis[mr]]**2 - p**2)*lis[mr]
        cur += (2*p+1)*lis[mr]
    while mr>r:
        p = cnt[lis[mr]]
        cnt[lis[mr]] -= 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur -= (2*p -1)*lis[mr]
        mr-=1
    ans[i] = cur
for i in ans: print(i)


# Heap

In [ ]:
# point to be NOTED
# 0 base index left child -> 2*i+1, right child -> 2*i+2 (i is parent)
# and parent -> (i-1)//2 (i is child)

from debug import debug

def left(i):
    return 2*i+1

def right(i):
    return 2*i+2

def valid(i, n):
    return (left(i)<=n-1), (right(i)<=n-1)

def verify(heap):
    n = len(heap)
    for i in range(n):
        l,r = valid(i, n)
        if r:
            if heap[right(i)]<heap[i]:
                print(False)
                debug(root=f"{i} -> {heap[i]}", c___=f"{right(i)} -> {heap[right(i)]}", child="right")
                return False
        if l:
            if heap[left(i)]<heap[i]:
                print(False)
                debug(root=f"{i} -> {heap[i]}", c___=f"{left(i)} -> {heap[left(i)]}", child="left")
                return False
    print(True)
    return True


def insert(heap, data):
    heap.append(data)
    n = len(heap)
    i = n-1
    while heap[i]<heap[i//2] and i!=0:
        heap[i], heap[i//2] = heap[i//2], heap[i]
        i = i//2

def delete(heap):
    if heap:
        val = heap[0]
        heap[0] = heap[-1]
        heap.pop()
        n = len(heap)
        i = 0
        while(i<n):
            l,r = valid(i, n)
            if r:
                index = left(i) if heap[left(i)]<=heap[right(i)] else right(i)
                if heap[i]>heap[index]:
                    heap[i], heap[index] = heap[index], heap[i]
            elif l:
                index = left(i)
                if heap[i]>heap[index]:
                    heap[i], heap[index] = heap[index], heap[i]
            else:
                break
            i = index
        return val
    else:
        return None


def insertion_arr(arr):
    n = len(arr)
    for j in range(1,n):
        i = j
        while arr[i]<arr[i//2] and i!=0:
            arr[i], arr[i//2] = arr[i//2], arr[i]
            i = i//2
        # debug(i=i, arr=arr)


def heapify(arr):
    n = len(arr)
    for i in range(n):
        l,r = valid(i, n)
        if l and r:
            index = left(i) if arr[left(i)]<=arr[right(i)] else right(i)
            if arr[i]>arr[index]:
                arr[i], arr[index] = arr[index], arr[i]
        elif l:
            index = left(i)
            if arr[i]>arr[index]:
                arr[i], arr[index] = arr[index], arr[i]
        elif r:
            index = right(i)
            if arr[i]>arr[index]:
                arr[i], arr[index] = arr[index], arr[i]

arr = [20, 21, 27, 29, 29]

insertion_arr(arr)
print(arr)
verify(arr)

# insert(arr, 7)
# print(arr)
# verify(arr)

print(delete(arr))
print(arr)
verify(arr)

# t = []
# insert(t, 0)
# print(t)

# Binary Index Tree

In [2]:
# from debug import debug
from math import ceil, log2

# root of a node x is x - ((x)&(-x))
# So first set bit of a number is for eg. b110 is b10, and for b1000 is b1000, for b110100 is b100.
# this first set bit can be easily calulated with (x)&(-x) operation.
# so root of node is first set bit subtracted from the node.
# 4 - 4 = 0 so root of 4 is 0(FSB of 4 = 4)
# 6 - 2 = 4 so root of 6 is 2(FSB of 6 = 2)
# https://www.hackerearth.com/practice/notes/binary-indexed-tree-or-fenwick-tree/
# go to above site for more recollection.


lis = [3, 2, 4, 5, 1, 1, 5, 3]
n = len(lis)
b = [0]*(n+1)

# update function takes only delta(differnce from previous data to curr changed data)
# so do not pass the changed value rather pass changed_value-previous_value.
def update(b, delta, index):
    index+=1
    while (index < len(b)):
        b[index] += delta;
        index += ((index)&(-index))

def presum(b, index):
    ans = 0
    index+=1
    while index>0:
        ans += b[index]
        index -= ((index)&(-index))
    return ans

for i in range(n):
    update(b, lis[i], i)

# IMPORTANT: DO NOT FORGET TO UPDATE INITIAL LIST, AND UPDATE THE LIST AFTER YOU CALL THE 
# UPDATE FUNCTION AND NOT INSIDE UPDATE FUNCTION.

if __name__ == "__main__":
    print(b)
    print(presum(b, 3)-presum(b, 0))


[0, 3, 5, 4, 14, 1, 2, 5, 24]
11


# Sparse Table

In [3]:
# from debug import debug
from math import ceil, log2
inf = int(1e3)

lis = [7,2,3,0,5,10,3,12,18]
n = len(lis)

def table(lis, inf = int(1e10)):
    n = len(lis)
    k = int(log2(n))+1
    grid = [[inf for _ in range(k)] for _ in range(n)]
    for i in range(n): grid[i][0] = lis[i]
    i = 0; j = 1
    while j<k:
        i = 0
        while i+(1<<j)-1<n:
            grid[i][j] = min(grid[i][j-1], grid[i+(1<<(j-1))][j-1])
            i+=1
        j+=1
    return grid

def querry(table, a, b):
    s = log2(b-a+1)
    if int(s) == ceil(s):
        return table[a][int(s)]
    s = int(s)
    return min(table[a][s], table[b-(1<<s)+1][s])

if __name__ == "__main__":
    t = table(lis)

# Trie

In [4]:
# from debug import debug
I = lambda x: ord(x)-ord('a')
C = lambda x: chr(x+ord('a'))
class node:
    def __init__(self, value=""):
        self.value = value
        self.words = 0
        self.prefixes = 0
        self.edges = [None]*26  # can use dict which is also very convenient

def addWord(root, word):
    if len(word) == 0: root.words += 1
    else:
        root.prefixes += 1
        first = word[0]
        if root.edges[I(first)] is None: root.edges[I(first)] = node(first)
        addWord(root.edges[I(first)], word[1:])

def countWords(root, word):
    if len(word) == 0: return root.words
    elif root.edges[I(word[0])] is None: return 0
    else: return countWords(root.edges[I(word[0])], word[1:])

def countPrefixes(root, word):
    if len(word) == 0: return root.prefixes+root.words
    elif root.edges[I(word[0])] is None: return 0
    else: return countPrefixes(root.edges[I(word[0])], word[1:])

def dfs(node):
    if node is not None:
        print(node.value, node.words, node.prefixes)
        for nodes in node.edges:
            if nodes is not None: dfs(nodes)


root = node()
# strings = ["hola","konnichiwa","henlo","bonjour","adios", "adidas"]
strings = ["ab","abab","c","cb"]
for string in strings: addWord(root, string)
s = "ababc"
print(countPrefixes(root, "c"))


2


# Aho Corsaik

In [11]:
# from debug import debug
from collections import deque

# word = input().strip()
# words = [input().strip() for _ in range(int(input()))]
words = ["ab", "abab", "c", "cb"]
word = "ababc"
word += "$"
class Node:
    def __init__(self, value=""):
        self.value = value
        self.end = 0
        self.d = dict()
        self.f = None

root = Node("#")

# making a trie...
def fill(root, w):
    if w == "": root.end = 1
    else:
        if w[0] not in root.d: root.d[w[0]] = Node(w[0])
        fill(root.d[w[0]], w[1:])

def find(root, w):
    if w == "": return root
    elif w[0] not in root.d: return None
    else: return find(root.d[w[0]], w[1:])

def dfs(root):
    print(root.value, root.end, end=" ")
    if root.f is not None: print(root.f.value)
    else: print()
    for i in root.d.values(): dfs(i)

for i in words: fill(root, i)

# making fail links...
q = deque()
root.f = root
for i in root.d.values():
    i.f = root
    q.append(i)

while q:
    node = q.popleft()
    for child in node.d.values():
        if node.f: child.f = find(node.f, child.value)
        else: child.f = find(root, child.value)
        q.append(child)

# dfs(root)
ans = 0
temp = root
for i in word:
    if temp.end: ans+=1
    if i in temp.d: temp = temp.d[i]
    elif temp.f is not None and i in temp.f.d:
        if temp.f.end: ans+=1
        temp = temp.f.d[i]
    elif i in root.d: temp = root.d[i]
    else: temp = root
print(ans)


3


# Segment Tree 

In [21]:
# point to be NOTED
# 0 base index left child -> 2*i+1, right child -> 2*i+2 (i is parent)
# and parent -> (i-1)//2 (i is child)

#this is 1-based index to ez
from math import log2, ceil

lis = [1,2,3,4,5,2,3,5,8,9,0]
# build
p = len(lis)
n = 2**ceil(log2(p))
lis = lis + [0]*(n-p)
tree = [0]*(2*n+1)
for i in range(n): tree[i+n] = lis[i]
for i in range(n-1, 0, -1): tree[i] = tree[2*i] + tree[2*i + 1]

# update
def update(pos, val):
    pos += n-1
    tree[pos] = val
    while pos>1:
        tree[pos//2] = tree[pos] + tree[pos^1]
        pos >>= 1
        
#querry
def querry(l, r):
    l+=n-1; r+=n-1
    ans = 0
    while l<=r:
        if (l&1): ans += tree[l]; l+=1
        if (not r&1): ans += tree[r]; r-=1
        l>>=1; r>>=1
    return ans
        
# to debug
def check(lis):
    for k in range(len(lis)):
        update(k+1, k**2)
        lis[k] = k**2
        for i in range(n):
            for j in range(i+1):
                if sum(lis[i:j+1]) != querry(i+1, j+1): 
                    print(sum(lis[i:j+1]))
                    print(querry(i+1, j+1))
                    print("NO")

42


# Segment Tree - Lazy

In [3]:
#this is 1-based index to ez

from debug import *
from math import log2, ceil
max_size = 2*int(15)
tree = [0]*(max_size)
lazy = [0]*(max_size)

#build
def build(lis):
    p = len(lis)
    n = 2**ceil(log2(p))
    for i in range(n-p): lis.append(0)
    for i in range(n): tree[i+n] = lis[i]
    for i in range(n-1, 0, -1): tree[i] = tree[2*i] + tree[2*i+1] # bit addition, after shifting for sure 0 th postion is 0 so adding 1 there only

# query
def query(node, start, end, l, r):
    if lazy[node]:
        dif = lazy[node]
        lazy[node] = 0
        tree[node] += dif*(end-start+1)
        if start != end: lazy[2*node] += dif; lazy[2*node + 1] += dif

    if start>r or end<l: return 0

    if start >= l and end <= r: return tree[node]

    mid = (start+end)//2
    return query(2*node, start, mid, l, r) + query(2*node+1, mid+1, end, l, r)

# update
def updateRange(node, start, end, l ,r, val):
    if lazy[node]:
        dif = lazy[node]
        lazy[node] = 0
        tree[node] += dif*(end-start+1)
        if start != end: lazy[2*node] += dif; lazy[2*node + 1] += dif

    if start>r or end<l: return

    if start >= l and end <= r:
        print(tree[node], val*(end-start+1))
        tree[node] += val*(end-start+1)
        if start != end: lazy[2*node] += val; lazy[2*node + 1] += val
        return

    mid = (start+end)//2
    updateRange(2*node, start, mid, l, r, val)
    updateRange(2*node+1, mid+1, end, l, r, val)
    tree[node] = tree[2*node] + tree[2*node + 1]

if __name__ == "__main__":
    lis = [1,2,3,4,5,6,7]
    n = len(lis)
    build(lis)
    updateRange(1, 1, n, 1, 1, 2) 
    print(query(1, 1, n, 1, n))

1 2
30


# AVL Tree

In [1]:
# AVL Tree
# Rotations are important; if in Doubt GFG is the best.
class Node:
    def __init__(self, val):
        self.val = val
        self.right = None
        self.left = None
        self.height = 1

class AVLTree:
    def __init__(self, lis=None):
        self.root = None
        if lis:
            for i in lis: self.insert(i)

    def insert(self, data):
        self.root = self._insert(data, self.root)
        
    def delete(self, data):
        self.root = self._delete(data, self.root)
        
    def _insert(self, data, root):
        # Null case
        if root == None: 
            root = Node(data)
            return root
        
        # Procceding towards correct position of data.
        if root.val < data: root.right = self._insert(data, root=root.right)
        else: root.left = self._insert(data, root=root.left)
        
        # updating the height.
        root.height = 1+max(self.getHeight(root.right), self.getHeight(root.left))
        balance = self.getBalance(root)
        
        # checking balance of node and performing appropiate rotation.
        
        # left left
        if balance > 1 and data < root.left.val: return self.rotateRight(root)
        
        # right right 
        if balance < -1 and data > root.right.val: return self.rotateLeft(root)
        
        # left right 
        if balance > 1 and data > root.left.val:
            root.left = self.rotateLeft(root.left)
            return self.rotateRight(root)
        
        # right left
        if balance <-1 and data < root.right.val:
            root.right = self.rotateRight(root.right)
            return self.rotateLeft(root)        
        return root

    def _delete(self, data, root=-1):  
        # Null case.
        if root is None: return root
        # standard BST deletion
        if root.val > data: root.left = self._delete(data, root=root.left)
        if root.val < data: root.right = self._delete(data, root=root.right)
        else:
            if root.left is None:
                temp = root.right
                root = None
                return temp
            if root.right is None:
                temp = root.left
                root = None
                return temp
            succ = root.right
            while succ.left: succ = succ.left
            root.val = succ.val
            root.right = self._delete(succ.val, root=root.right)
        # BST deletion ends.
        
        if root is None: return root
        # updating height.
        root.height = 1+max(self.getHeight(root.right), self.getHeight(self.root.left))
        
        balance = self.getBalance(root)
        # same as in _insertions check for appropiate rotations.
        
        if balance > 1 and self.getBalance(root.left) >= 0: return self.rotateRight(root)
        
        if balance < -1 and self.getBalance(root.left) <= 0: return self.rotateLeft(root)
        
        if balance > 1 and self.getBalance(root.right) < 0:
            root.left = self.rotateLeft(root.left)
            return self.rotateRight(root)
        
        if balance < -1 and self.getBalance(root.right) > 0:
            root.right = self.rotateRight(root.right)
            return self.rotateLeft(root)
        return root

    def rotateRight(self, z):
        y = z.left
        T2 = y.right
        y.right = z
        z.left = T2
        z.height = 1+max(self.getHeight(z.left), self.getHeight(z.right))
        y.height = 1+max(self.getHeight(y.left), self.getHeight(y.right))
        return y

    def rotateLeft(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1+max(self.getHeight(z.left), self.getHeight(z.right))
        y.height = 1+max(self.getHeight(y.left), self.getHeight(y.right))
        return y

    def getHeight(self, root):
        if root is None: return 0
        return root.height

    def getBalance(self, root):
        if root is None: return 0
        return self.getHeight(root.left) - self.getHeight(root.right)

    def search(self, data, lower=True):
        node = self.root
        less = great = None
        def find(root, data, less, great):
            if root:
                if root.val < data:
                    less = root
                    if root.right: return find(root.right, data, less, great)
                    # return root
                if root.val > data:
                    great = root
                    if root.left: return find(root.left, data, less, great)
                    # return root
                if root.val == data: return root
            return less if lower else great
        return find(node, data, less, great)


    def inOrder(self):
        self._inOrder()
        print()

    def _inOrder(self, root=-1):
        if root == -1: root = self.root
        if root:
            self._inOrder(root=root.left)
            print(root.val, end=" ")
            self._inOrder(root=root.right)

    def debug(self):
        from collections import deque
        q = deque([self.root])
        lis = []
        while q:
            node = q.popleft()
            if node: lis.append(node.val)
            else: lis.append("None")
            if node: q.append(node.left)
            if node: q.append(node.right)
        n = len(lis)
        for i in range(n):
            if lis[i] != "None":
                left = "None"
                right = "None"
                if 2*i + 1<n: left = lis[2*i+1]
                if 2*i + 2<n: right = lis[2*i+2]
                print(lis[i], left, right)

lis = [10, 20, 30, 40, 50, 60, 70, 70, 70]
tree = AVLTree(lis)
tree.insert(80)
tree.inOrder()
print(tree.root.val)
tree.delete(40)
tree.inOrder()
print(tree.root.val)


10 20 30 40 50 60 70 70 70 80 
60
10 20 30 50 70 70 70 80 
70


In [39]:
s = AVLTree([1,2,3,4,5,6,7,8,9])
print(s.search(5, lower=False).val)

5


# Difference Array

In [3]:
#code
lis = [1,2,3,4,5,3,4,5,6]
n = len(lis)
# making diff array
diff = [0]*n
diff[0] = lis[0]
for i in range(1, n): diff[i] = lis[i]-lis[i-1]
#query handing, 0-based indexing
def update(l, r, x):
    diff[l] += x
    if r+1<len(diff): diff[r+1] -= x
        
def convertingBack():
    lis[0] = diff[0]
    for i in range(1, n): lis[i] = lis[i-1]+diff[i]

# correction check 
q = [(0, 3, 3), (4, 5, -1), (6, 8, 2), (5, 5, -2)]
A = [*lis]
for a, b, x in q:
    update(a, b, x)
    for i in range(a, b+1):
        A[i] += x
convertingBack()
print(lis)
print(A)

[4, 5, 6, 7, 4, 0, 6, 7, 8]
[4, 5, 6, 7, 4, 0, 6, 7, 8]


# Median finding Algorithm

1. subdividing into set of 5.
2. collecting medians of those small sets(simple sort).
3. finding pivot, if around 5 then simple sort, else use recursion.
4. quick sorting technique to partition around pivot.
5. compare the rank(position) of pivot to what rank you want to find.

In [3]:
def MedOfMed(lis,k):
    sets = [sorted(lis[i:i+5]) for i in range(0, len(lis), 5)]  # 1
    medians = [v[len(v)//2] for v in sets] # 2
    pivot = sorted(medians)[len(medians)//2] if len(medians)<=5 else MedOfMed(medians, len(medians)//2) # 3
    low = [i for i in lis if i<pivot] # 4
    high = [i for i in lis if i>pivot]
    r = len(low) #5
    if k<r: return MedOfMed(low, k)
    elif k>r: return MedOfMed(high, k-r-1)
    else: return pivot
lis = range(10)[:][::-1]
MedOfMed(lis, 5)

5